In [2]:
import meshio as mio
import meshplot as mp
import numpy as np
from utils import *
import torch
from tqdm import tqdm

In [3]:
# m = mio.read("../meshes/sphere/sphere3.msh")
m = mio.read("../meshes/armadillo/armadillop4.msh")

order = 3 if m.cells[-1].type == 'tetra20' else 4
T = m.cells[-1].data.astype(int)
V = m.points

In [4]:
ti = 80
# 
rV, rT = sample_tet(8)

TT = T[ti, :]
pts = V[TT]*100

target = np.array([80., 80., 70.])

opt, vv = inver_gmapping(order, rV[0, :], pts, target)

  3%|▎         | 28/1000 [00:02<01:07, 14.34it/s]


In [6]:
print(type(opt), type(pts))

print(gmapping(order, opt, pts))
print(gmapping_energy(order, opt, pts, target))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[80.00000004 79.99999948 69.99999971]
3.6041587584165737e-13


In [ ]:


# remapping = [0, 1, 2, 3,   4, 5,  6, 7,   8, 9,    11, 10,  15, 14,  13, 12,    16, 17, 19, 18]
rV, rT = sample_tet(6)

ppp = []
ttt = []
tot = 0


for ti in tqdm(range(T.shape[0])):
    TT = T[ti, :]
    pts = V[TT]

    img_p = gmapping(order, rV, pts)

    ppp += img_p.tolist()
    ttt += (rT+tot).tolist()
    tot+=img_p.shape[0]


ppp = np.array(ppp)
ttt = np.array(ttt, dtype=int)

p_uni, _, inverse = np.unique(ppp, return_index=True, return_inverse=True, axis=0)
t_uni = np.array([inverse[ttt[:, 0]], inverse[ttt[:, 1]], inverse[ttt[:, 2]], inverse[ttt[:, 3]]]).transpose()

p = mp.plot(p_uni, t_uni, shading={"wireframe": True})
p.add_points(V, shading={"point_size": 0.05})

In [ ]:
ttt

In [ ]:
x.tolist